In [ ]:
#noetbook utilizado para a aula sobre árvore de decisão e floresta randômica

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [4]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("ArvoreDecisao").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
diretorioArvore="/content/gdrive/MyDrive/Arquivos-MLib/Arquivos/cap4/iris_bezdekIris.csv"  #diretório que contém o arquivo a ser utilizado

In [7]:
#lendo arquivos armazenados CSV com o esquema definido
df_iris = spark.read.format('csv').options(inferSchema = True,header='false',delimiter=',').load(diretorioArvore)

In [8]:
df_iris.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)



In [ ]:
df_iris.show(5)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 5 rows



In [9]:
#modificando o nome das colunas existentes no cabeçalho 
df_iris = df_iris.selectExpr("_c0 as sep_len", "_c1 as sep_wid", "_c2 as pet_len", "_c3 as pet_wid", "_c4 as label")

In [ ]:
df_iris.show(5)

+-------+-------+-------+-------+-----------+
|sep_len|sep_wid|pet_len|pet_wid|      label|
+-------+-------+-------+-------+-----------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|
+-------+-------+-------+-------+-----------+
only showing top 5 rows



Conhecendo o Banco de Dados

In [10]:
#encontrando as "estatísticas"
df_iris.describe(['sep_len','sep_wid','pet_len','pet_wid']).show()

+-------+------------------+-------------------+------------------+------------------+
|summary|           sep_len|            sep_wid|           pet_len|           pet_wid|
+-------+------------------+-------------------+------------------+------------------+
|  count|               150|                150|               150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|
|    min|               4.3|                2.0|               1.0|               0.1|
|    max|               7.9|                4.4|               6.9|               2.5|
+-------+------------------+-------------------+------------------+------------------+



In [ ]:
#definindo a visão do dataframe para ser utilizado como uma tabela pelo SQL
df_iris.createOrReplaceTempView("irisTable")

In [ ]:
spark.sql('select * from irisTable').show()

+-------+-------+-------+-------+-----------+
|sep_len|sep_wid|pet_len|pet_wid|      label|
+-------+-------+-------+-------+-----------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|
|    5.4|    3.9|    1.7|    0.4|Iris-setosa|
|    4.6|    3.4|    1.4|    0.3|Iris-setosa|
|    5.0|    3.4|    1.5|    0.2|Iris-setosa|
|    4.4|    2.9|    1.4|    0.2|Iris-setosa|
|    4.9|    3.1|    1.5|    0.1|Iris-setosa|
|    5.4|    3.7|    1.5|    0.2|Iris-setosa|
|    4.8|    3.4|    1.6|    0.2|Iris-setosa|
|    4.8|    3.0|    1.4|    0.1|Iris-setosa|
|    4.3|    3.0|    1.1|    0.1|Iris-setosa|
|    5.8|    4.0|    1.2|    0.2|Iris-setosa|
|    5.7|    4.4|    1.5|    0.4|Iris-setosa|
|    5.4|    3.9|    1.3|    0.4|Iris-setosa|
|    5.1|    3.5|    1.4|    0.3|Iris-setosa|
|    5.7|    3.8|    1.7|    0.3|I

Iniciando o Processo de Construção e Aplicação da Árvore de Decisão

In [11]:
from pyspark.ml.linalg import Vectors  #biblioteca que contém funções de Algebra Linear
from pyspark.ml.feature import VectorAssembler #biblioteca que contém as funções para a construção de vetores

In [12]:
#criando o vetor de características
vector_assembler = VectorAssembler(inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"],outputCol="features")
df_temp = vector_assembler.transform(df_iris)
df_temp.show(5)

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 5 rows



In [13]:
#removendo as colunas que não serão utilizadas
df_menor = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df_menor.show(5)

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
|Iris-setosa|[4.6,3.1,1.5,0.2]|
|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+-----------------+
only showing top 5 rows



In [14]:
#aplicando as transformações para a coluna label
from pyspark.ml.feature import StringIndexer  #cria o 'vetor' para cada uma das classes existentes na coluna label

l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")  #cria o objeto para a codificação
df_final = l_indexer.fit(df_menor).transform(df_menor)  #aplica a transformação

In [ ]:
df_final.show(5)

+-----------+-----------------+----------+
|      label|         features|labelIndex|
+-----------+-----------------+----------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
+-----------+-----------------+----------+
only showing top 5 rows



In [15]:
#dividindo entre dados de treinamento e teste
(train, test) = df_final.randomSplit([0.7, 0.3])

In [16]:
test.show(5)

+-----------+-----------------+----------+
|      label|         features|labelIndex|
+-----------+-----------------+----------+
|Iris-setosa|[4.4,2.9,1.4,0.2]|       0.0|
|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|Iris-setosa|[4.9,3.1,1.5,0.2]|       0.0|
|Iris-setosa|[5.0,3.0,1.6,0.2]|       0.0|
+-----------+-----------------+----------+
only showing top 5 rows



Definindo o Algoritmo

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier  #biblioteca para o algoritmo da árvore de decisão
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  #utilizada para encontrar as métricas de desempenho

In [18]:
modeloArvore = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features")  #definindo o modelo
model = modeloArvore.fit(train)  #aplicando o treinamento

In [19]:
#realizando a previsão
predictions = model.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows



In [20]:
#encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [ ]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo =   0.9545454545454546


Aplicação da Floresta Randômica

In [21]:
from pyspark.ml.classification import RandomForestClassifier  #classificador para o randomForest


In [22]:
modeloRF = RandomForestClassifier(labelCol="labelIndex",featuresCol="features", numTrees=10)  #define o modelo
modelRF = modeloRF.fit(train)

In [23]:
#realizando a previsão
predictions = modelRF.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows



In [24]:
#encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [25]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo =   0.8913043478260869


In [26]:
print(modelRF)

RandomForestClassificationModel (uid=RandomForestClassifier_94520a716b25) with 10 trees
